## Import Necessary Libraries

In [5]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.processing import ProcessingInput, ProcessingOutput

region = boto3.session.Session().region_name

role = get_execution_role()a

## Build and Push The Processing Container to ECR

In [10]:
import boto3

account_id = boto3.client("sts").get_caller_identity().get("Account")
ecr_repository = "sagemaker-processing-container"
tag = ":latest"

uri_suffix = "amazonaws.com"
if region in ["cn-north-1", "cn-northwest-1"]:
    uri_suffix = "amazonaws.com.cn"
processing_repository_uri = "{}.dkr.ecr.{}.{}/{}".format(
    account_id, region, uri_suffix, ecr_repository + tag
)

# Create ECR repository and push docker image
!docker build -t $ecr_repository .
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $processing_repository_uri
!docker push $processing_repository_uri

Sending build context to Docker daemon  2.048kB
Step 1/5 : FROM python:3.7-slim-buster
3.7-slim-buster: Pulling from library/python

b615420c: Pulling fs layer 
f1d7987a: Pulling fs layer 
b5347801: Pulling fs layer 
ec4af258: Pulling fs layer 
Digest: sha256:19150e60094fd825c5d6785af9a5a6a75c89edee0f1bd73db3646741b627651f[5A
Status: Downloaded newer image for python:3.7-slim-buster
 ---> ad65429aadd2
Step 2/5 : RUN apt-get update && apt-get install -y python3-opencv
 ---> Running in b44386634713
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian-security buster/updates InRelease [73.5 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:4 http://deb.debian.org/debian buster/main amd64 Packages [7911 kB]
Get:5 http://deb.debian.org/debian-security buster/updates/main amd64 Packages [337 kB]
Get:6 http://deb.debian.org/debian buster-updates/main amd64 Packages [8788 B]
Fetched 8508 kB in 1s (5946 kB/s)
Reading package

In [9]:
processing_repository_uri

'973994155064.dkr.ecr.us-east-2.amazonaws.com/sagemaker-processing-container:latest'

The `ScriptProcessor` class lets you run a command inside this container, which you can use to run your own script.

In [12]:
!aws s3 cp ./code/preprocessing.py s3://smart-city-conf-demo/code/

upload: code/preprocessing.py to s3://smart-city-conf-demo/code/preprocessing.py


In [1]:
!aws s3 ls s3://smart-city-conf-demo/processing_output